# Introduction to SSA in Scalpel

## What is the Static Single Assignment (SSA)

**SSA** is a means of structuring the IR (intermediate representation) in compiling design, where requires each variable is defined exactly once. It can be used to generate use-def chain by renaming each variable assignment with different names, which can track precisely how data is flowing in the program. Some Algorithms can be improved by the application of SSA like dead code elimination, value numbering and constant propagation. 
Generally, a variable may be assigned more than once in a program which will result in many instances. Taking the following as an example. On the left, the first line of y seems useless since the x only uses the second assignment of y. However, a program needs to find the reaching definition to achieve this. On the right example which is an SSA form where all of them are immediate:

y = 1 $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$y1 = 1

y = 2 $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$y2 = 2

x = y $\;\;\;\;\;\;$  ->$\;\;\;\;\;\;$x = y2

## Let's install Scalpel first

Use the command in your virtual environment to install Scalpel.
```console
pip install python-scalpel
```

### Import essential libraries

In [1]:
import os
import sys
import ast
import astor
import unittest

### Import MNode and SSA API

In [2]:
from scalpel.core.mnode import MNode
from scalpel.SSA.const import SSA

### Let's have an example code

In [3]:
code_str = """
a = 0
b = 10
if b>0:
    a = a+b
else:
    a = 10
print(a)
"""

In this example, the variable a has two possible values. By applying phi function in SSA, an initial assignment to a has been added so that instance of a will have two corresponding values.

### Generate CFGs for given source file
The function mnode.gen_ast() is to build an AST tree for source code first, then using cfg building method to generate a CFG for the function.

In [4]:
mnode = MNode("local") # create file name for given source code
mnode.source = code_str
mnode.gen_ast() # build abstract syntax tree
cfg = mnode.gen_cfg() 
res = cfg.build_visual('pdf')
res.render('test-output/DNN', view=True)

NODE_LABEL______________________
#1
a = 0
b = 10
if b > 0:

___________________________________________
state:Assign(targets=[Name(id='a', ctx=Store())], value=Constant(value=0, kind=None), type_comment=None)
Assign(targets=[Name(id='b', ctx=Store())], value=Constant(value=10, kind=None), type_comment=None)
If(test=Compare(left=Name(id='b', ctx=Load()), ops=[Gt()], comparators=[Constant(value=0, kind=None)]), body=[Assign(targets=[Name(id='a', ctx=Store())], value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())), type_comment=None)], orelse=[Assign(targets=[Name(id='a', ctx=Store())], value=Constant(value=10, kind=None), type_comment=None)])
{'load_model'}
False
NODE_LABEL______________________
#2
a = a + b

___________________________________________
state:Assign(targets=[Name(id='a', ctx=Store())], value=BinOp(left=Name(id='a', ctx=Load()), op=Add(), right=Name(id='b', ctx=Load())), type_comment=None)
{'load_model'}
False
NODE_LABEL______________________

'test-output\\DNN.pdf'

### Compute SSA and build SSA graph
Constant value and alias pairs are generated by computing cfg. Please note the function m_ssa.compute_SSA() will return two dictionaries.

In [5]:
m_ssa = SSA()
ssa_results, const_dict = m_ssa.compute_SSA(cfg.functioncfgs['texas_holdem'])

The key in the first dictionary is the block numbers in given CFG, the value is the statement result of the corresponding block. Here is the output of the results. It can be seen that in the last block, variable a takes two possible values.

In [7]:
for block_id, stmt_res in ssa_results.items():
    print("These are the results for block ", block_id)
    print(stmt_res)

These are the results for block  1
[{}, {}, {'b': {0}}]
These are the results for block  2
[{'a': {0}, 'b': {0}}]
These are the results for block  4
[{}]
These are the results for block  3
[{'print': set(), 'a': {1, 2}}]


The second dictionary is the global constant values for the numbered identifiers. There are many types of constant values.

In [8]:
for name, value in const_dict.items():
    print(name, value)

('a', 0) <_ast.Constant object at 0x0000021EB1C36940>
('b', 0) <_ast.Constant object at 0x0000021EB1C36880>
('a', 1) <_ast.BinOp object at 0x0000021EB1C36790>
('a', 2) <_ast.Constant object at 0x0000021EB1C36AC0>


In [10]:
print(ssa_results)

{1: [{}, {}, {}, {}, {}, {'mnist.load_data': set(), 'mnist': {0}}, {'x_train.astype': set(), 'x_train': {0}}, {'x_test.astype': set(), 'x_test': {0}}, {'to_categorical': {0}, 'y_train': {0}}, {'to_categorical': {0}, 'y_test': {0}}, {'np.expand_dims': set(), 'np': {0}, 'x_train': {1}}, {'np.expand_dims': set(), 'np': {0}, 'x_test': {1}}, {'models.Sequential': set(), 'models': {0}, 'layers.Conv2D': set(), 'layers': {0}, 'layers.MaxPooling2D': set(), 'layers.Flatten': set(), 'layers.Dense': set()}, {'model.compile': set(), 'model': {0}}, {'model.fit': set(), 'model': {0}, 'x_train': {2}, 'y_train': {1}, 'x_test': {2}, 'y_test': {1}}, {'model.evaluate': set(), 'model': {0}, 'x_test': {2}, 'y_test': {1}}, {'print': set(), 'test_acc': {0}}, {}, {'model.save': set(), 'model': {0}}, {'load_model': {0}}, {'loaded_model.predict': set(), 'loaded_model': {0}, 'x_test': {2}}, {}, {}, {}, {}, {}, {'load_model': {1}}, {'mnist.load_data': set(), 'mnist': {1}}, {'x_test.astype': set(), 'x_test': {3}}, 

### Alias analysis
The alias pairs can be implemented, since the const_dict stores all the constant values of variables. If the assigned value is an ast.Name type, the alias pairs can be found as follow.

In [16]:
alias_name_pairs = []
for name, value in const_dict.items():
    print(name, value)
    if isinstance(value, ast.Name):
        print(name)
        alias_name_pairs.append((name, value.id))

('x_train', 1) <_ast.BinOp object at 0x000001D81A4F2E20>
('x_test', 1) <_ast.BinOp object at 0x000001D81A4F2DF0>
('y_train', 1) <_ast.Call object at 0x000001D81A5050A0>
('y_test', 1) <_ast.Call object at 0x000001D81A505220>
('x_train', 2) <_ast.Call object at 0x000001D81A505340>
('x_test', 2) <_ast.Call object at 0x000001D81A5054F0>
('model', 0) <_ast.Call object at 0x000001D81A505700>
('test_loss', 0) <_ast.Call object at 0x000001D81A509BB0>
('test_acc', 0) <_ast.Call object at 0x000001D81A509BB0>
('loaded_model', 0) <_ast.Call object at 0x000001D81A50E0A0>
('all_predictions', 0) <_ast.Call object at 0x000001D81A50E1C0>
('all_predictions', 1) <_ast.Constant object at 0x000001D81A50E2E0>
('loaded_model', 1) <_ast.Call object at 0x000001D81A50E5B0>
('x_test', 4) <_ast.BinOp object at 0x000001D81A50E8E0>
('x_test', 5) <_ast.Call object at 0x000001D81A50EA30>
('texas_holdem', 0) <_ast.FunctionDef object at 0x000001D81A50EBB0>
('generate_hand', 0) <_ast.FunctionDef object at 0x000001D81A50